# Imports

In [1]:
import os
import zipfile

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import scipy.io


# Parameters

In [2]:
original_dirname = "../../data/original_datasets/jasper_ridge"
original_hsi_basename = "jasperRidge2_F224_2.mat"
original_groundtruth_basename = "GroundTruth.zip"

converted_dirname = "../../data/datasets/jasper_ridge"
converted_basename = "jasper_ridge_full.npz"

# Function Definitions

In [3]:
def mkdir(dir):
    try: 
        os.makedirs(dir)
    except OSError:
        if not os.path.isdir(dir):
            raise

# Data Loading

## HSI Data

In [4]:
print(original_hsi_basename)
filename = os.path.join(original_dirname, original_hsi_basename)
matlab_data = scipy.io.loadmat(filename)
print(matlab_data.keys())

n_rows = matlab_data["nRow"].flatten()[0]
n_cols = matlab_data["nCol"].flatten()[0]
n_bands = matlab_data["nBand"].flatten()[0]
hsi_data = np.transpose(matlab_data["Y"].reshape(n_bands, n_rows, n_cols))

print(f"hsi_data: {hsi_data.shape}")

jasperRidge2_F224_2.mat
dict_keys(['__header__', '__version__', '__globals__', 'Region', 'SlectBands', 'nRow', 'nCol', 'nBand', 'Y', 'maxValue'])
hsi_data: (100, 100, 224)


## Abundance Maps and Endmembers

In [5]:
print(original_groundtruth_basename)
filename = os.path.join(original_dirname, original_groundtruth_basename)

with zipfile.ZipFile(filename, "r") as zip_fp:
    print(zip_fp.namelist())
    matlab_data = scipy.io.loadmat( zip_fp.open("GroundTruth/end4.mat") )
print(matlab_data.keys())

endmembers = matlab_data["M"].transpose()
abundance_maps = np.transpose(
    matlab_data["A"].reshape(len(endmembers), n_rows, n_cols),
    (2,1,0)
)
labels = ["Trees", "Water", "Soil", "Road"]
colors = ["forestgreen", "royalblue", "saddlebrown", "dimgray"]

print(f"    endmembers: {endmembers.shape}")
print(f"abundance_maps: {abundance_maps.shape}")
print(f"        labels: {labels}")

GroundTruth.zip
['GroundTruth/', 'GroundTruth/end4.mat', 'GroundTruth/end4_Abundance.fig', 'GroundTruth/end4_Materials.fig']
dict_keys(['__header__', '__version__', '__globals__', 'cood', 'M', 'A'])
    endmembers: (4, 198)
abundance_maps: (100, 100, 4)
        labels: ['Trees', 'Water', 'Soil', 'Road']


# Spatial Resolution

In [6]:
resolution = "nm"
selected_bands = np.arange(1, n_bands+1, dtype=int)
spectral_bands = np.linspace(380, 2500, 225)

# Save Converted Data

In [7]:
mkdir(converted_dirname)
print(converted_basename)
filename = os.path.join(converted_dirname, converted_basename)
np.savez(
    filename,
    hsi_data=hsi_data,
    endmembers=endmembers,
    abundance_maps=abundance_maps,
    labels=labels,
    colors=colors,
    spectral_bands=spectral_bands,
    selected_bands=selected_bands,
    resolution=resolution
)
print(filename)

jasper_ridge_full.npz
../../data/datasets/jasper_ridge/jasper_ridge_full.npz
